In [16]:
! pip3 install bayesian_optimization
from bayes_opt import BayesianOptimization
from bayes_opt import SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
import pandas as pd 
import catboost as cb 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

Defaulting to user installation because normal site-packages is not writeable


In [17]:
train_values = pd.read_csv("train_values.csv", index_col='building_id')
train_values1 = pd.read_csv('Archivos auxiliares KNN/train_values_short1.csv', index_col='building_id')

In [18]:
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')

In [280]:
importances_superestructures = [0.11087213, 0.24814224, 0.06244005, 0.03015171, 0.08679662,
       0.1561809 , 0.0585955 , 0.04264239, 0.07171384, 0.11541968,
       0.01704496]

superestructures = ['has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other']

reemplazos = {}
for i in range(len(superestructures)):
    reemplazos[superestructures[i]] = {1:importances_superestructures[i]}    

In [281]:
train_values.replace(reemplazos, inplace=True)

In [282]:
train_values['hasnt_secondary'] = (train_values['has_secondary_use'] != True) * 1

In [283]:
importances_secondary_uses = [0.19752775, 0.27044932, 0.21866441, 0.02404361, 0.00476977,
       0.00504387, 0.00198575, 0.00234236, 0.00102254, 0.02242508,
       0.25172554]
secondary_uses =  ['has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
       'has_secondary_use_health_post', 'has_secondary_use_gov_office',
       'has_secondary_use_use_police', 'has_secondary_use_other', 'hasnt_secondary']

reemplazos = {}
for i in range(len(secondary_uses)):
    reemplazos[secondary_uses[i]] = {1:importances_secondary_uses[i]}    

In [284]:
train_values.replace(reemplazos, inplace=True)

In [285]:
train_values ['superestructure'] = train_values\
                .loc[:, superestructures].idxmax(axis='columns')
train_values.drop(columns=superestructures, inplace=True)


train_values ['secondary_use'] = train_values\
                .loc[:, secondary_uses].idxmax(axis='columns')
train_values.drop(columns=secondary_uses, inplace=True)

In [286]:
train_values.drop(columns=["has_secondary_use"], inplace=True)
train_values.drop(columns=[ "geo_level_1_id","geo_level_2_id","geo_level_3_id"], inplace=True)

In [287]:
train_values = train_values.merge(train_values1[["min_geo_id", "max_geo_id"]],right_index=True, left_index=True)

In [294]:
train_values.to_csv('train_values_catboost.csv', index=True)

In [288]:
for col in train_values.columns:
    print(train_values[col].value_counts())

2    156623
3     55617
1     40441
4      5424
5      2246
6       209
7        39
9         1
8         1
Name: count_floors_pre_eq, dtype: int64
10     38896
15     36010
5      33697
20     32182
0      26041
25     24366
30     18028
35     10710
40     10559
50      7257
45      4711
60      3612
80      3055
55      2033
70      1975
995     1390
100     1364
65      1123
90      1085
85       847
75       512
95       414
120      180
150      142
200      106
110      100
105       89
125       37
115       21
140        9
130        9
180        7
160        6
170        6
175        5
135        5
145        3
190        3
195        2
165        2
155        1
185        1
Name: age, dtype: int64
6      42013
7      36752
5      32724
8      28445
9      22199
       ...  
82         1
80         1
78         1
75         1
100        1
Name: area_percentage, Length: 84, dtype: int64
5     78513
6     46477
4     37763
7     35465
3     25957
8     13902
2      9305
9      

In [11]:
# A excepcion de los geo_levels todas las columnas tienen pocos valores por lo que 
# se usar todas como categoricas
categoricas = []
for col in train_values.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col)
        categoricas.append(train_values.columns.get_loc(col))
        train_values[col] = train_values[col].astype("category")

In [290]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   count_floors_pre_eq     260601 non-null  category
 1   age                     260601 non-null  category
 2   area_percentage         260601 non-null  category
 3   height_percentage       260601 non-null  category
 4   land_surface_condition  260601 non-null  category
 5   foundation_type         260601 non-null  category
 6   roof_type               260601 non-null  category
 7   ground_floor_type       260601 non-null  category
 8   other_floor_type        260601 non-null  category
 9   position                260601 non-null  category
 10  plan_configuration      260601 non-null  category
 11  legal_ownership_status  260601 non-null  category
 12  count_families          260601 non-null  category
 13  superestructure         260601 non-null  category
 14 

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.2, random_state=10) 

In [14]:
model = cb.CatBoostClassifier(loss_function='MultiClass', eval_metric='TotalF1', iterations=100) 

In [15]:
model.fit(X_train, y_train, categoricas)

Learning rate set to 0.5
0:	learn: 0.7009538	total: 531ms	remaining: 52.5s
1:	learn: 0.7218660	total: 982ms	remaining: 48.1s
2:	learn: 0.7279061	total: 1.4s	remaining: 45.1s
3:	learn: 0.7334521	total: 1.79s	remaining: 43.1s
4:	learn: 0.7357449	total: 2.22s	remaining: 42.2s
5:	learn: 0.7389619	total: 2.64s	remaining: 41.4s
6:	learn: 0.7396439	total: 3.05s	remaining: 40.6s
7:	learn: 0.7412938	total: 3.49s	remaining: 40.1s
8:	learn: 0.7424586	total: 3.9s	remaining: 39.4s
9:	learn: 0.7428130	total: 4.31s	remaining: 38.8s
10:	learn: 0.7440533	total: 4.71s	remaining: 38.1s
11:	learn: 0.7443327	total: 5.13s	remaining: 37.6s
12:	learn: 0.7453833	total: 5.54s	remaining: 37.1s
13:	learn: 0.7457692	total: 5.96s	remaining: 36.6s
14:	learn: 0.7456499	total: 6.47s	remaining: 36.7s
15:	learn: 0.7462045	total: 6.96s	remaining: 36.5s
16:	learn: 0.7464531	total: 7.38s	remaining: 36s
17:	learn: 0.7469428	total: 7.86s	remaining: 35.8s
18:	learn: 0.7476034	total: 8.33s	remaining: 35.5s
19:	learn: 0.7479908

In [296]:
pred = model.predict(X_test) 
from sklearn.metrics import f1_score
f1_score(pred, y_test, average='micro')

0.7564321482703709

# Tunning

In [19]:
train_values = pd.read_csv('train_values_catboost.csv', index_col='building_id')

In [26]:
# Pruebo otra funcion de perdida
model = cb.CatBoostClassifier(loss_function='MultiClassOneVsAll', eval_metric='TotalF1', iterations=100) 

In [4]:
# A excepcion de los geo_levels todas las columnas tienen pocos valores por lo que 
# se usar todas como categoricas
categoricas = []
for col in train_values.columns:
    if ((col != 'min_geo_id') & (col != 'max_geo_id')):
        categoricas.append(col)
        categoricas.append(train_values.columns.get_loc(col))
        train_values[col] = train_values[col].astype("category")

In [27]:
X_train, X_test, y_train, y_test = train_test_split(train_values, train_labels, test_size=0.2, random_state=10) 

In [28]:
model.fit(X_train, y_train, categoricas)

0:	learn: 0.7034333	total: 452ms	remaining: 44.7s
1:	learn: 0.7155420	total: 916ms	remaining: 44.9s
2:	learn: 0.7220559	total: 1.33s	remaining: 42.9s
3:	learn: 0.7221535	total: 1.73s	remaining: 41.5s
4:	learn: 0.7222003	total: 2.14s	remaining: 40.6s
5:	learn: 0.7223115	total: 2.6s	remaining: 40.8s
6:	learn: 0.7246373	total: 3.02s	remaining: 40.1s
7:	learn: 0.7222333	total: 3.49s	remaining: 40.1s
8:	learn: 0.7249711	total: 4.05s	remaining: 40.9s
9:	learn: 0.7246856	total: 4.64s	remaining: 41.8s
10:	learn: 0.7266033	total: 5.14s	remaining: 41.6s
11:	learn: 0.7262348	total: 5.55s	remaining: 40.7s
12:	learn: 0.7263313	total: 5.96s	remaining: 39.9s
13:	learn: 0.7267787	total: 6.49s	remaining: 39.9s
14:	learn: 0.7266296	total: 7.06s	remaining: 40s
15:	learn: 0.7267857	total: 7.57s	remaining: 39.7s
16:	learn: 0.7277278	total: 7.97s	remaining: 38.9s
17:	learn: 0.7281003	total: 8.38s	remaining: 38.2s
18:	learn: 0.7276539	total: 8.79s	remaining: 37.5s
19:	learn: 0.7280934	total: 9.2s	remaining: 

In [29]:
pred = model.predict(X_test) 
from sklearn.metrics import f1_score
f1_score(pred, y_test, average='micro')

0.7460332687400472

Los resultados son peores asi que se usara la de antes

In [14]:
def funcion_a_optimizar(iterations):
    pool = cb.Pool(data=train_values,
                  label=train_labels,
                  cat_features=categoricas)
    params = {"loss_function":'MultiClass', "eval_metric":'TotalF1', "iterations":round(iterations),
                                  "od_pval":0.01, "od_type":"IncToDec", 'silent':True, "random_seed":2021,
             "task_type":'GPU'}
    return float(cb.cv(pool, params, fold_count=10).tail(1)['test-TotalF1-mean'])    

In [9]:
funcion_a_optimizar(600)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0.7511100971795747

In [10]:
funcion_a_optimizar(700)

0.7513669173122761

In [11]:
funcion_a_optimizar(800)

0.7515741042568524

In [15]:
funcion_a_optimizar(1000)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

CatBoostError: catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
funcion_a_optimizar(1000)

In [ ]:
funcion_a_optimizar(750)

In [ ]:
funcion_a_optimizar(850)

In [ ]:
funcion_a_optimizar(650)

In [ ]:
limites = {iterations:(300,500)}

In [ ]:
optimizer = BayesianOptimization(
    f=funcion_a_optimizar,
    random_state=1300,
    verbose=5,
    pbounds=limites,
    bounds_transformer=SequentialDomainReductionTransformer(),
)

#load_logs(optimizer, "./anteriores.json")
logger = JSONLogger(path="./iterations.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.probe({"iterations":500}, lazy=True)

In [ ]:
optimizer.probe({"iterations":300}, lazy=True)

In [ ]:
optimizer.maximize(
    init_points=5,
    n_iter=40,
)

In [25]:
    pool = cb.Pool(data=train_values,
                  label=train_labels,
                  cat_features=categoricas)
    params = {"loss_function":'MultiClass', "eval_metric":'TotalF1', "iterations":2000,
                                  "od_pval":0.01, "od_type":"IncToDec", 'silent':True, "random_seed":2021}
    resultados = cb.cv(pool, params, fold_count=10)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

KeyboardInterrupt: 